In [1]:
%matplotlib inline 

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
def read_df(filename, valtype):
    df = pd.read_csv(filename, low_memory=False, dtype=valtype)
    return df

# Read Dependent feature data

In [4]:
base = './data/'

In [5]:
y_train = pd.read_csv(base+'fin_train_indeps.csv', encoding='utf8')
y_train = y_train.drop(['Unnamed: 0'], axis=1)
y_train.sample()

,review_scores_rating
16896,94.0


In [6]:
y_test = pd.read_csv(base+'fin_test_indeps.csv', encoding='utf8')
y_test = y_test.drop(['Unnamed: 0'], axis=1)
y_test.sample()

,review_scores_rating
21597,100.0


# Read independent feature data

In [7]:
num_train = pd.read_csv(base+'fin_num_train_deps.csv')
num_train = num_train.drop(['Unnamed: 0'], axis=1)
num_test = pd.read_csv(base+'fin_num_test_deps.csv')
num_test = num_test.drop(['Unnamed: 0'], axis=1)

cat_train = pd.read_csv(base+'fin_cat_train_deps.csv')
cat_train = cat_train.drop(['Unnamed: 0'], axis=1)
cat_test = pd.read_csv(base+'fin_cat_test_deps.csv')
cat_test = cat_test.drop(['Unnamed: 0'], axis=1)

amen_train = pd.read_csv(base+'fin_amen_train_deps.csv')
amen_train = amen_train.drop(['Unnamed: 0'], axis=1)
amen_test = pd.read_csv(base+'fin_amen_test_deps.csv')
amen_test = amen_test.drop(['Unnamed: 0'], axis=1)

comb_train = pd.read_csv(base+'fin_comb_train_deps.csv')
comb_train = comb_train.drop(['Unnamed: 0'], axis=1)
comb_test = pd.read_csv(base+'fin_comb_test_deps.csv')
comb_test = comb_test.drop(['Unnamed: 0'], axis=1)

# Split Test data into Holdout and Validation sets

In [8]:
np.random.seed(1234)
idxs = np.random.randint(0, y_test.shape[0], y_test.shape[0]//2)

In [9]:
y_validation = y_test.iloc[~y_test.index.isin(idxs)]
y_holdout = y_test.iloc[idxs]

In [10]:
num_validation = num_test.iloc[~num_test.index.isin(idxs)]
num_holdout = num_test.iloc[idxs]

cat_validation = cat_test.iloc[~cat_test.index.isin(idxs)]
cat_holdout = cat_test.iloc[idxs]

amen_validation = amen_test.iloc[~amen_test.index.isin(idxs)]
amen_holdout = amen_test.iloc[idxs]

comb_validation = comb_test.iloc[~comb_test.index.isin(idxs)]
comb_holdout = comb_test.iloc[idxs]

In [11]:
# Objective: Implement a wrapper for vecstack
# InputL scikit model instances and training data
# Output: Predictions

import vecstack
import xgboost
import dill as pickle
from copy import deepcopy 
from vecstack import StackingTransformer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib

class VecstackRunner():

    def __init__(self, X, y, Xt, yt, l1_estimators, l2_estimator, prfx='', metric='rmse', regression=True, nfolds=10, verbose=2):
        self.X = X
        self.y = y
        self.Xt = Xt
        self.yt = yt
        self.yp = None
        self.ytp = None
        self.l1 = l1_estimators
        self.l2 = l2_estimator
        self.prfx = prfx
        self.metname = metric
        self.nf = nfolds
        self.v = verbose
        self.is_reg = regression
        self.stack = None
    
    def mse(self, actual, pred):
        return mean_squared_error(actual, pred)
    
    def rsq(self, actual, pred):
        return r2_score(actual, pred)
    
    def get_metric_calc(self, actual, pred):
        if 'rmse' == self.metname:
            return self.mse(actual, pred)
        elif 'r2' == self.metname:
            return self.rsq(actual, pred)
    
    def build_stack(self):
        self.stack = StackingTransformer(self.l1, 
                                        regression=self.is_reg, 
                                        shuffle=True, 
                                        n_folds=self.nf, 
                                        metric=self.get_metric_calc, 
                                        verbose=self.v)
    
    def fit_stack(self):
        return self.stack.fit(self.X, self.y)
    
    def transform(self):
        self.X = self.stack.transform(self.X)
        self.Xt = self.stack.transform(self.Xt)
    
    def fit_l2(self):
        self.l2.fit(self.X, self.y)
    
    def predict(self):
        self.yp = self.l2.predict(self.X)
        self.ytp = self.l2.predict(self.Xt)
    
    def calculate_error(self, calc=None):
        if calc:
            y_err = calc(self.y, self.yp)
            yp_err = calc(self.y, self.ytp)
        else:
            y_err = self.get_metric_calc(self.y, self.yp)
            yt_err = self.get_metric_calc(self.yt, self.ytp)
            
        return [('Train err', y_err), ('Test err', yt_err)]
    
    def save_to_disk(self):
        sname = self.prfx+'_vecstack_stack.pkl'
        cname = self.prfx+'_vecstack_clf.pkl'
        
        with open('/tmp/'+sname, 'wb') as f:
            pickle.dump(self.stack,f)
        
        with open('/tmp/'+cname, 'wb') as f:
            pickle.dump(self.l2, f)
    
    def run(self):
        self.build_stack()
        self.fit_stack()
        self.transform()
        self.fit_l2()
        self.predict()
        self.save_to_disk()
        return self.calculate_error()

In [12]:
from sklearn.linear_model import ElasticNet, Lasso, Ridge
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR


en = ElasticNet(alpha=0.001)
lass = Lasso(alpha=0.001, fit_intercept=False, max_iter=10000)
ridge = Ridge(normalize=False, alpha=0.1, max_iter=10000)
gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.01, 
                               subsample=0.75, max_depth=15)
ab = AdaBoostRegressor(n_estimators=100, learning_rate=0.01)
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.05, 
                           gamma=0, subsample=0.5, 
                           max_depth=15, objective='reg:linear')
randf = RandomForestRegressor(n_estimators=100, min_samples_split=10)
svr_rbf = SVR(kernel='rbf')

In [13]:
from copy import deepcopy

In [14]:
estimators = [
                ('en', en), 
                ('lass', lass), 
                ('ridge', ridge),
                ('gb', gb), 
                ('ab', ab), 
                ('randf', randf), 
             ]

prf = 'num'

with open(base+prf+'_vecstack_stack.pkl', 'rb') as f:
    ts = pickle.load(f)

with open(base+prf+'_vecstack_clf.pkl', 'rb') as f:
    tc = pickle.load(f)
    
Xt = ts.transform(num_train)
Xp = tc.predict(Xt)
y = y_train

Xvt = ts.transform(num_test)
Xvp = tc.predict(Xvt)
yv = y_test

print '####################'
print 'Column: %s' % prf
print 'Train: '
print 'MSE: %s' % mean_squared_error(y, Xp)
print 'R2: %s' % r2_score(y, Xp) 
print 'Test: '
print 'MSE: %s' % mean_squared_error(yv, Xvp)
print 'R2: %s' % r2_score(yv, Xvp)
print '####################\n'

/Users/jhabib/miniconda2/lib/python2.7/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator ElasticNet from version 0.18.1 when using version 0.19.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/jhabib/miniconda2/lib/python2.7/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator Lasso from version 0.18.1 when using version 0.19.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/jhabib/miniconda2/lib/python2.7/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator Ridge from version 0.18.1 when using version 0.19.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/jhabib/miniconda2/lib/python2.7/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator GradientBoostingRegressor from version 0.18.1 when using version 0.19.2. This might lead to breaking c

Train set was detected.
Transforming...

estimator  0: [en: ElasticNet]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    model from fold  6: done
    model from fold  7: done
    model from fold  8: done
    model from fold  9: done
    ----
    DONE

estimator  1: [lass: Lasso]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    model from fold  6: done
    model from fold  7: done
    model from fold  8: done
    model from fold  9: done
    ----
    DONE

estimator  2: [ridge: Ridge]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    model from fold  6: done
    model from fold  7: done
    model from fold  

AttributeError: 'GradientBoostingRegressor' object has no attribute 'n_features_'

In [15]:
estimators = [
                ('en', en), 
                ('lass', lass), 
                ('ridge', ridge),
                ('gb', gb), 
                ('ab', ab), 
                ('randf', randf), 
             ]

prf = 'cat'

with open(base+prf+'_vecstack_stack.pkl', 'rb') as f:
    ts = pickle.load(f)

with open(base+prf+'_vecstack_clf.pkl', 'rb') as f:
    tc = pickle.load(f)

Xt = ts.transform(cat_train)
Xp = tc.predict(Xt)
y = y_train

Xvt = ts.transform(cat_test)
Xvp = tc.predict(Xvt)
yv = y_test

print '####################'
print 'Column: %s' % prf
print 'Train: '
print 'MSE: %s' % mean_squared_error(y, Xp)
print 'R2: %s' % r2_score(y, Xp) 
print 'Test: '
print 'MSE: %s' % mean_squared_error(yv, Xvp)
print 'R2: %s' % r2_score(yv, Xvp)
print '####################\n'

IOError: [Errno 2] No such file or directory: './data/cat_vecstack_stack.pkl'

In [16]:
estimators = [
                ('en', en), 
                ('lass', lass), 
                ('ridge', ridge),
                ('gb', gb), 
                ('ab', ab), 
                ('randf', randf), 
             ]

prf = 'amen'

with open(base+prf+'_vecstack_stack.pkl', 'rb') as f:
    ts = pickle.load(f)

with open(base+prf+'_vecstack_clf.pkl', 'rb') as f:
    tc = pickle.load(f)

Xt = ts.transform(amen_train)
Xp = tc.predict(Xt)
y = y_train

Xvt = ts.transform(amen_test)
Xvp = tc.predict(Xvt)
yv = y_test

print '####################'
print 'Column: %s' % prf
print 'Train: '
print 'MSE: %s' % mean_squared_error(y, Xp)
print 'R2: %s' % r2_score(y, Xp) 
print 'Test: '
print 'MSE: %s' % mean_squared_error(yv, Xvp)
print 'R2: %s' % r2_score(yv, Xvp)
print '####################\n'

Train set was detected.
Transforming...

estimator  0: [en: ElasticNet]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    model from fold  6: done
    model from fold  7: done
    model from fold  8: done
    model from fold  9: done
    ----
    DONE

estimator  1: [lass: Lasso]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    model from fold  6: done
    model from fold  7: done
    model from fold  8: done
    model from fold  9: done
    ----
    DONE

estimator  2: [ridge: Ridge]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    model from fold  6: done
    model from fold  7: done
    model from fold  

AttributeError: 'GradientBoostingRegressor' object has no attribute 'n_features_'

In [0]:
estimators = [
                ('en', en), 
                ('lass', lass), 
                ('ridge', ridge),
                ('gb', gb), 
                ('ab', ab), 
                ('randf', randf), 
             ]
prf = 'comb'

with open(base+prf+'_vecstack_stack.pkl', 'rb') as f:
    ts = pickle.load(f)

with open(base+prf+'_vecstack_clf.pkl', 'rb') as f:
    tc = pickle.load(f)

Xt = ts.transform(comb_train)
Xp = tc.predict(Xt)
y = y_train

Xvt = ts.transform(comb_test)
Xvp = tc.predict(Xvt)
yv = y_test

print '####################'
print 'Column: %s' % prf
print 'Train: '
print 'MSE: %s' % mean_squared_error(y, Xp)
print 'R2: %s' % r2_score(y, Xp) 
print 'Test: '
print 'MSE: %s' % mean_squared_error(yv, Xvp)
print 'R2: %s' % r2_score(yv, Xvp)
print '####################\n'